In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # Suppress SettingWithCopyWarning
import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # for ignoring the FutureWarning

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv("C:\\Users\\SRN\\OneDrive\\Desktop\\Python\\projects\\olympics_till_2016\\athlete_events.csv")
region_df=pd.read_csv("C:\\Users\SRN\\OneDrive\\Desktop\\Python\\projects\\olympics_till_2016\\noc_regions.csv")

In [3]:
df.isnull().sum()

df.shape

ID             0
Name           0
Sex            0
Age         9474
Height     60171
Weight     62875
Team           0
NOC            0
Games          0
Year           0
Season         0
City           0
Sport          0
Event          0
Medal     231333
dtype: int64

(271116, 15)

In [4]:
df=df[df['Season']=='Summer']

In [5]:
df=df.merge(region_df,on='NOC',how='left')

In [6]:
df.drop_duplicates(inplace=True)

In [7]:
dummy_medals=pd.get_dummies(df['Medal'])

In [8]:
df=pd.concat([df,dummy_medals],axis=1)

In [9]:
medal_df=df.drop_duplicates(subset=['Team','NOC','Games','City','Sport','Event','Medal'])

In [10]:
pred_df=medal_df[['region','Sport','Age','Sex','Height','Weight','Medal']]


In [11]:
num=pred_df.select_dtypes(include=['number']).columns
cat=pred_df.select_dtypes(include=['object','category']).columns
print('\nNumeric columns: ',num)
print('\nCategoric columns: ',cat)


Numeric columns:  Index(['Age', 'Height', 'Weight'], dtype='object')

Categoric columns:  Index(['region', 'Sport', 'Sex', 'Medal'], dtype='object')


In [12]:
#filling the missing values in numerical columns
#with the median value of the respective columns
pred_df[num]=pred_df[num].fillna(pred_df[num].median())

In [13]:
pred_df['region'].fillna('Independent',inplace=True) 

In [14]:
#replacing the nan value in medal column with 0
pred_df['Medal'].fillna('None', inplace=True)

In [15]:
pred_df['Medal'].unique()

array(['None', 'Gold', 'Bronze', 'Silver'], dtype=object)

In [16]:
pred_df.isnull().sum()

pred_df.shape

region    0
Sport     0
Age       0
Sex       0
Height    0
Weight    0
Medal     0
dtype: int64

(105636, 7)

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
import pandas as pd

# Assuming pred_df is your DataFrame containing the data
# Convert 'Sex' column to categorical codes
pred_df['Sex'] = pred_df['Sex'].astype('category').cat.codes

# Encoding the 'Medal' column
medal_mapping = {'None': 0, 'Gold': 1, 'Silver': 2, 'Bronze': 3}
pred_df['Medal'] = pred_df['Medal'].map(medal_mapping)

# Convert 'region' and 'Sport' columns to one-hot encoding
X = pd.get_dummies(pred_df[['region', 'Sport', 'Age', 'Sex', 'Height', 'Weight']])
y = pred_df['Medal']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initializing Gaussian Naive Bayes model
gnb = GaussianNB()

# Training the model
gnb.fit(X_train, y_train)

# Taking user input
region = input("Enter region: ")
sport = input("Enter sport: ")
age = float(input("Enter age: "))
sex = input("Enter sex (female/male): ")
height = float(input("Enter height (in cm): "))
weight = float(input("Enter weight (in kg): "))

# Convert user input to one-hot encoding
input_data = pd.get_dummies(pd.DataFrame({'region': [region], 'Sport': [sport], 'Age': [age], 'Sex': [sex], 'Height': [height], 'Weight': [weight]}))

# Ensure input_data has the same columns as X_train
input_data = input_data.reindex(columns=X_train.columns, fill_value=0)

# Making predictions
prediction = gnb.predict(input_data)

# Inverse mapping to get predicted medal name
predicted_medal_name = list(medal_mapping.keys())[list(medal_mapping.values()).index(prediction[0])]

# Print user input and predicted medal
print("\nUser Input:")
print("Region:", region)
print("Sport:", sport)
print("Age:", age)
print("Sex:", sex)
print("Height:", height)
print("Weight:", weight)

# Print predicted medal
print("\nPredicted Medal:", predicted_medal_name)


GaussianNB()

ValueError: could not convert string to float: ''

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initializing Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Training the model
rf.fit(X_train, y_train)

# Taking user input
region = input("Enter region: ")
sport = input("Enter sport: ")
age = float(input("Enter age: "))
sex = input("Enter sex (female/male): ")
height = float(input("Enter height (in cm): "))
weight = float(input("Enter weight (in kg): "))

# Convert user input to one-hot encoding
input_data = pd.get_dummies(pd.DataFrame({'region': [region], 'Sport': [sport], 'Age': [age], 'Sex': [sex], 'Height': [height], 'Weight': [weight]}))

# Ensure input_data has the same columns as X_train
input_data = input_data.reindex(columns=X_train.columns, fill_value=0)

# Making predictions
prediction = rf.predict(input_data)

# Inverse mapping to get predicted medal name
predicted_medal_name = list(medal_mapping.keys())[list(medal_mapping.values()).index(prediction[0])]

# Print user input and predicted medal
print("\nUser Input:")
print("Region:", region)
print("Sport:", sport)
print("Age:", age)
print("Sex:", sex)
print("Height:", height)
print("Weight:", weight)

# Print predicted medal
print("\nPredicted Medal:", predicted_medal_name)


RandomForestClassifier(random_state=42)


User Input:
Region: India
Sport: Swimming
Age: 27.0
Sex: M
Height: 160.0
Weight: 60.0

Predicted Medal: None


In [ ]:
# Training set accuracy
train_accuracy = rf.score(X_train, y_train)
print("Training Set Accuracy:", train_accuracy)

# Testing set accuracy
test_accuracy = rf.score(X_test, y_test)
print("Testing Set Accuracy:", test_accuracy)


Training Set Accuracy: 0.9591399630804184
Testing Set Accuracy: 0.8306039379023097
